## Setup and Installation

In [1]:
# Install required packages
%pip install -r requirements.txt

# Additional packages for LLM API interaction
%pip install requests

# Import necessary libraries
import os
import sys
import requests
import time
import logging
import argparse
from typing import Optional

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Create directories
os.makedirs('utils', exist_ok=True)
os.makedirs('results/part_2', exist_ok=True)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. Connecting to the Hugging Face API

The Hugging Face Inference API provides access to many language models. We'll use models that are available on the free tier.

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model once
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

def query(payload):
    """
    Send a query to the Hugging Face API

    Args:
        payload: Dictionary containing the query parameters

    Returns:
        The API response
    """
    text_to_process = payload.get("inputs", "")
    tokenized_inputs = tokenizer(text_to_process, return_tensors="pt")
    generated_tokens = model.generate(**tokenized_inputs)
    decoded_output = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return decoded_output

# Test the query function
test_payload = {"inputs": "What are the symptoms of diabetes?"}
response = query(test_payload)
print(response)

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-06 18:39:04.151168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749235144.698934    4272 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749235144.894902    4272 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749235146.152766    4272 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000

a swollen rectum


## 2. Creating Simple Chat Scripts

Your task is to create two simple scripts that interact with the Hugging Face API:

1. A basic one-off chat script (`utils/one_off_chat.py`)
2. A contextual conversation script (`utils/conversation.py`)

### One-Off Chat Script

Create a script that handles independent interactions (each prompt/response is separate):

In [ ]:
# utils/one_off_chat.py

import requests
import argparse
import os
from transformers import pipeline


def get_response(prompt, model_name="google/flan-t5-base"):
    """
    Get a response from the model

    Args:
        prompt: The prompt to send to the model
        model_name: Name of the model to use
        api_key: API key for authentication (optional for some models)

    Returns:
        The model's response
    """
    # TODO: Implement the get_response function
    # Set up the API URL and headers
    # Create a payload with the prompt
    # Send the payload to the API
    # Extract and return the generated text from the response
    # Handle any errors that might occur
    API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {api_key}"} if api_key else {}
    payload = {"inputs": prompt}
    try:
        generator = pipeline("text2text-generation", model=model_name)
        response = generator(prompt, max_new_tokens=100)[0]['generated_text']
        return response
    except Exception as e:
        return f"Error: {e}"

def run_chat():
    """Run an interactive chat session"""
    print("Welcome to the Simple LLM Chat! Type 'exit' to quit.")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break

        # TODO: Get response from the model
        # Print the response
        response = get_response(user_input, model_name=model_name, api_key=api_key)
        print(f"Bot: {response}")

def main():
    run_chat()

if __name__ == "__main__":
    main()

Welcome to the Simple LLM Chat! Type 'exit' to quit.


### Contextual Conversation Script

Create a script that maintains conversation history:

In [ ]:
# utils/conversation.py

from transformers import pipeline
import requests
import argparse
import os

def get_response(prompt, history=None, model_name="google/flan-t5-base", api_key=None, history_length=3):
    """
    Get a response from the model using conversation history

    Args:
        prompt: The current user prompt
        history: List of previous (prompt, response) tuples
        model_name: Name of the model to use
        api_key: API key for authentication
        history_length: Number of previous exchanges to include in context

    Returns:
        The model's response
    """
    # Initialize history if None
    if history is None:
        history = []

    # Keep only the last `history_length` exchanges
    history = history[-history_length:]

    # Format the full prompt with history
    full_prompt = ""
    for i, (user_msg, bot_msg) in enumerate(history):
        full_prompt += f"User: {user_msg}\nBot: {bot_msg}\n"
    full_prompt += f"User: {prompt}\nBot:"

    try:
        generator = pipeline("text2text-generation", model=model_name)
        response = generator(full_prompt, max_new_tokens=100)[0]['generated_text']
        return response.strip()
    except Exception as e:
        return f"Error: {e}"
    
def run_chat():
    print("Welcome to the Contextual LLM Chat! Type 'exit' to quit.")
    history = []

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break

        response = get_response(user_input, history)
        history.append((user_input, response))
        print("Bot:", response)

def main():
    run_chat()

if __name__ == "__main__":
    main()

usage: colab_kernel_launcher.py [-h] [--model MODEL] [--api_key API_KEY]
                                [--history_length HISTORY_LENGTH]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-4ff23fe0-1956-4622-9454-92b479c134c4.json


SystemExit: 2

## 3. Testing and Evaluation

Create a script to test your chat implementations with specific healthcare questions.


In [ ]:
# utils/test_chat.py

import os
import csv
from pathlib import Path
import sys
sys.path.append('utils')

# Import our chat modules - since we're in the same directory
from one_off_chat import get_response as get_one_off_response
# Optionally import the conversation module if testing that too
# from conversation import get_response as get_contextual_response

def test_chat(questions, model_name="google/flan-t5-base", api_key=None):
    """
    Test the chat function with a list of questions

    Args:
        questions: A list of questions to test
        model_name: Name of the model to use
        api_key: API key for authentication

    Returns:
        A dictionary mapping questions to responses
    """
    results = {}

    for question in questions:
        print(f"Testing question: {question}")
        # Get response using the one-off chat function
        response = get_one_off_response(question, model_name, api_key)
        results[question] = response

    return results

# List of healthcare questions to test
test_questions = [
    "What are the symptoms of gout?",
    "How is gout diagnosed?",
    "What treatments are available for gout?",
    "What lifestyle changes can help manage gout?",
    "What foods should be avoided with gout?"
]

def save_results(results, output_file="results/part_2/example.txt"):
    """
    Save the test results to a file

    Args:
        results: Dictionary mapping questions to responses
        output_file: Path to the output file
    """    
    Path(output_file).parent.mkdir(parents=True, exist_ok=True)
    with open(output_file, 'w') as f:
        # Write header
        f.write("# LLM Chat Tool Test Results\n\n")

        # Write usage examples
        f.write("## Usage Examples\n\n")
        f.write("```bash\n")
        f.write("# Run the one-off chat\n")
        f.write("python utils/one_off_chat.py\n\n")
        f.write("# Run the contextual chat\n")
        f.write("python utils/conversation.py\n")
        f.write("```\n\n")

        # Write test results
        f.write("## Test Results\n\n")
        f.write("```csv\n")
        f.write("question,response\n")

        for question, response in results.items():
            # Format the question and response for CSV
            q = question.replace(',', '').replace('\n', ' ')
            r = response.replace(',', '').replace('\n', ' ')
            f.write(f"{q},{r}\n")

        f.write("```\n")

# Run the test and save results
if __name__ == "__main__":
    results = test_chat(test_questions)
    save_results(results)
    print("Test results saved to results/part_2/example.txt")

ModuleNotFoundError: No module named 'one_off_chat'